# CROSS-ENCODER EXPERIMENT

* 새로운 라벨로 train 된 model과 기존 klue label로 train 된 model의 성능비교 실험  
* 전체 실험과정에선 epoch를 3으로 하고 10가지의 랜덤시드를 통한 평균값을 평가에 사용
* 본 예제 출력은 epoch1로 고정 

#### import libraries

In [1]:
# ! pip install sentence-transformers datasets

In [2]:
import numpy as np
import pandas as pd
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator, CEBinaryAccuracyEvaluator, CEBinaryClassificationEvaluator

from sentence_transformers.readers import InputExample




from tqdm.auto import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# 학습경과 모니터링하는 logger 초기화
from importlib import reload
reload(logging)

logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.DEBUG,
    handlers=[LoggingHandler()],
)

In [4]:
logging.info('This is an info message')

2022-12-20 17:45:47 - This is an info message


In [5]:
# 시드고정 
import torch
import random
import torch.backends.cudnn as cudnn


seed = 0

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)

### Load Model

#### cross-encoder 

In [23]:
# 예시 출력이기 때문에 epoch 1로 고정 

from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder = CrossEncoder("klue/roberta-base", num_labels=1)
num_epochs=1

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'class

In [24]:
train_batch_size = 32
model_save_path = "/home/sol3sts/train&dev_oglabel_cross_encoder_" + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

### Load Data (KLUE-STS)

### 기존 & 새로운 데이터 동시 로드 

* original label = labels.label
* changed label = new_label

* testset으로 사용할 eval은 변경된 라벨로 고정 
* trainset과 validationset만 기존라벨 / 변경된라벨 로 나눠 학습하여 성능비교 

In [8]:
train = pd.read_csv('/home/sol3sts/dataset/1209_train_dev&val제거.csv')
val = pd.read_csv('/home/sol3sts/dataset/1209_val.csv')
eval = pd.read_csv('/home/sol3sts/dataset/new_devset.csv')

In [9]:
train.head()

,index,guid,source,sentence1,sentence2,labels.label,new_label
0,0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.7,3.7
1,1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.0,0.0
2,2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.3,0.3
3,3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.6,0.6
4,4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.7,4.7


In [10]:
val.head()

,index,source,sentence1,sentence2,labels.label,new_label
0,9,airbnb-sampled,다음에 도쿄를 또 간다면 무조건 이 곳을 다시 찾을 겁니다.,독일을 다음에 또간다면 이숙소에 머물겁니다.,2.2,2.5
1,10,paraKQC-para,라디오 듣는 건 금지되어있어,라디오 듣는건 삼가주세요,2.5,3.0
2,25,paraKQC-para,집안에서 효율적으로 환기하고 싶을 때 뭐가 필요해?,발코니말고 집안에서 효율적으로 환기할 때 필요한 것은?,2.7,3.0
3,27,airbnb-sampled,신발 신고다니는 숙소라 불편하신 분들은 슬리퍼챙겨오시는게 좋을듯여,슬리퍼 등 실내화를 챙기시는게 좋을 것 같아요.,3.0,2.5
4,36,paraKQC-para,보일러는 구월말고 시월되기 전에 꼭 끝낼 수 있게 해줘,구월말고 시월 전에 보일러 점검 안 하면 큰일나.,2.5,3.0


In [11]:
len(train),len(val),len(eval)

(10992, 638, 557)

In [12]:
eval.head()

,index,source,sentence1,sentence2,new_label
0,0,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",4.9
1,1,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,1.4
2,2,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,1.3
3,3,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",3.7
4,4,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,3.0


In [13]:
# test data 변경된 라벨로 변경해주기 
eval= eval[['sentence1','sentence2','new_label']]
test = eval.rename(columns={'new_label':'labels.label'})
test.head()

,sentence1,sentence2,labels.label
0,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",4.9
1,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,1.4
2,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,1.3
3,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",3.7
4,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,3.0


## 기존 라벨과 새로운 라벨의 학습결과 비교

####  기존라벨(klue-original_trainset) 'labels.label'로 데이터 로드 


In [14]:
# 기존 데이터로 학습 
train = train.reset_index()
train = train[['sentence1','sentence2','labels.label']]

val = val.reset_index()
val = val[['sentence1','sentence2','labels.label']]

logging.info("기존 데이터 로드 ")

# original klue label trainset
train

2022-12-20 17:45:51 - 기존 데이터 로드 


,sentence1,sentence2,labels.label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.7
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.3
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.6
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.7
...,...,...,...
10987,일단 방은 사진이랑 똑같고 매우 깨끗합니다.,"우선, 방은 그림과 같고 매우 깨끗합니다.",4.0
10988,생일이 삼월이라는 친구 이름이 뭐야?,"운동하러 갈 시간은 밥 먹기 전이 좋아, 먹은 후가 좋아?",0.0
10989,그녀는 지속적으로 나에게 그녀가 만든 새로운 규칙을 매일 매일 이메일로 보냈다.,그녀는 자신이 만든 새로운 규칙을 매일 끊임없이 제게 이메일로 보냈습니다.,3.7
10990,개 1마리 고양이 3마리 너무 귀여워요!,개 한 마리와 고양이 세 마리는 정말 귀여워요!,4.7


In [15]:
train = train[['sentence1','sentence2','labels.label']]
val = val[['sentence1','sentence2','labels.label']]
test = test[['sentence1','sentence2','labels.label']]

In [16]:
# 모델에 입력형태로 바꿔주기 (train, val, test)

input_examples_train=[]
for i in tqdm(range(len(train))):
  sentence1 = train.values[i][0]
  sentence2 = train.values[i][1]
  score = train.values[i][2]/5.0 # 0-1사이로 정규화 
  input_examples_train.append(InputExample(texts=[sentence1, sentence2], label=score))

input_examples_val=[]
for i in tqdm(range(len(val))):
  sentence1 = val.values[i][0]
  sentence2 = val.values[i][1]
  score = val.values[i][2]/5.0
  input_examples_val.append(InputExample(texts=[sentence1, sentence2], label=score))

input_examples_test=[]
for i in tqdm(range(len(test))):
  sentence1 = test.values[i][0]
  sentence2 = test.values[i][1]
  score = test.values[i][2]/5.0
  input_examples_test.append(InputExample(texts=[sentence1, sentence2], label=score))

  0%|          | 0/10992 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/557 [00:00<?, ?it/s]

In [17]:
sts_train_examples = input_examples_train
sts_val_examples = input_examples_val
sts_test_examples = input_examples_test

### Evaluator 정의

In [18]:
## CROSS-ENCODER ##
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator


# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size, 
)

# Evaluator CEcorrelationEvaluator
logging.info("Creating cross-encoder evaluators")
val_evaluator = CECorrelationEvaluator.from_input_examples(
    sts_val_examples,
    name="sts-cross_encoder-val",
)


#Evaluator by sts-test
test_evaluator = CECorrelationEvaluator.from_input_examples(
    sts_test_examples,
    name="sts-cross_encoder-test",
)

2022-12-20 17:46:19 - Creating cross-encoder evaluators


### training

In [19]:
## CROSS-ENCODER ## 


# linear learning-rate warmup steps
warmup_steps = math.ceil(len(train_dataloader) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps)) # 학습 로그 표시 



# Training
cross_encoder.fit(
    train_dataloader = train_dataloader,
    evaluator=val_evaluator,
    epochs=num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    optimizer_params = {'lr':5e-5},
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    show_progress_bar = True
)

2022-12-20 17:46:19 - Warmup-steps: 2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/344 [00:00<?, ?it/s]

In [20]:
original_klue_cross_encoder = cross_encoder

In [25]:
oglabel_model_save_path = model_save_path
oglabel_model_save_path

'/home/sol3sts/train&dev_oglabel_cross_encoder_-2022-12-20_17-51-06'

### 새로운 데이터 (new_label)로 데이터 로드 



In [26]:
# 모델 초기화 

from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder = CrossEncoder("klue/roberta-base", num_labels=1)
num_epochs=1

train_batch_size = 32
model_save_path = "/home/sol3sts/train&dev_newlabel_cross_encoder_" + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'class

In [27]:
newlabel_model_save_path = model_save_path
newlabel_model_save_path

'/home/sol3sts/train&dev_newlabel_cross_encoder_-2022-12-20_17-51-31'

#### new_label로 trainset 교체 

In [28]:
train = pd.read_csv('/home/sol3sts/dataset/1209_train_dev&val제거.csv')
val = pd.read_csv('/home/sol3sts/dataset/1209_val.csv')
eval = pd.read_csv('/home/sol3sts/dataset/new_devset.csv')

train.head()

,index,guid,source,sentence1,sentence2,labels.label,new_label
0,0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.7,3.7
1,1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.0,0.0
2,2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.3,0.3
3,3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.6,0.6
4,4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.7,4.7


In [29]:
# 새로운 데이터로 학습 - new_label로 바꿔주기 

train = train.reset_index()
train = train[['sentence1','sentence2','new_label']]
train = train.rename(columns={'new_label':'labels.label'})


val = val.reset_index()
val = val[['sentence1','sentence2','new_label']]
val = val.rename(columns={'new_label':'labels.label'})

print(len(train))
logging.info("새로운 데이터 로드 ")

10992
2022-12-20 17:52:07 - 새로운 데이터 로드 


In [30]:
train.head()

,sentence1,sentence2,labels.label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.7
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.3
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.6
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.7


### 모델 입력 형태로 바꿔주기 (공통) 

In [31]:
# 분포 확인 
# fig, ax = plt.subplots(ncols=2,nrows=2, figsize=(10,10))


# sns.countplot(x = val['labels.label'],ax=ax[0,0],color='#FFB6C1')
# sns.countplot(x = test['labels.label'],ax=ax[0,1],color='#FFB6C1')
# sns.countplot(x = train['labels.label'],ax=ax[1,0],color='#FFB6C1')

# ax[0,0].set_title("validation data")
# ax[0,1].set_title("test data")
# ax[1,0].set_title('train_data')

# plt.show()

In [32]:
train = train[['sentence1','sentence2','labels.label']]
val = val[['sentence1','sentence2','labels.label']]
test = test[['sentence1','sentence2','labels.label']]

In [33]:
# 모델에 입력형태로 바꿔주기 (train, val, test)

input_examples_train=[]
for i in tqdm(range(len(train))):
  sentence1 = train.values[i][0]
  sentence2 = train.values[i][1]
  score = train.values[i][2]/5.0 # 0-1사이로 정규화 
  input_examples_train.append(InputExample(texts=[sentence1, sentence2], label=score))

input_examples_val=[]
for i in tqdm(range(len(val))):
  sentence1 = val.values[i][0]
  sentence2 = val.values[i][1]
  score = val.values[i][2]/5.0
  input_examples_val.append(InputExample(texts=[sentence1, sentence2], label=score))

input_examples_test=[]
for i in tqdm(range(len(test))):
  sentence1 = test.values[i][0]
  sentence2 = test.values[i][1]
  score = test.values[i][2]/5.0
  input_examples_test.append(InputExample(texts=[sentence1, sentence2], label=score))

  0%|          | 0/10992 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/557 [00:00<?, ?it/s]

In [34]:
sts_train_examples = input_examples_train
sts_val_examples = input_examples_val
sts_test_examples = input_examples_test

In [35]:
sts_val_examples[0].texts, sts_val_examples[0].label

(['다음에 도쿄를 또 간다면 무조건 이 곳을 다시 찾을 겁니다.', '독일을 다음에 또간다면 이숙소에 머물겁니다.'], 0.5)

In [36]:
len(input_examples_train), len(input_examples_val), len(input_examples_test)

(10992, 638, 557)

### Evaluator 정의

In [37]:
## CROSS-ENCODER ##
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator


# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size, 
)

# Evaluator CEcorrelationEvaluator
logging.info("Creating cross-encoder evaluators")
val_evaluator = CECorrelationEvaluator.from_input_examples(
    sts_val_examples,
    name="sts-cross_encoder-val",
)


#Evaluator by sts-test
test_evaluator = CECorrelationEvaluator.from_input_examples(
    sts_test_examples,
    name="sts-cross_encoder-test",
)

2022-12-20 17:52:52 - Creating cross-encoder evaluators


###  training

In [38]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Dec 20 17:52:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.88       Driver Version: 418.88       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P4000        On   | 00000000:3B:00.0 Off |                  N/A |
| 46%   34C    P8     5W / 105W |   6476MiB /  8119MiB |      0%      Default |
+---

In [39]:
# GPU 할당 변경하기
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

print ('# Current cuda device: ', torch.cuda.current_device()) # check

# Current cuda device:  1


In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 1
Count of using GPUs: 2


In [41]:
## CROSS-ENCODER ## 


# linear learning-rate warmup steps
warmup_steps = math.ceil(len(train_dataloader) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps)) # 학습 로그 표시 



# Training
cross_encoder.fit(
    train_dataloader = train_dataloader,
    evaluator=val_evaluator,
    epochs=num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    optimizer_params = {'lr':5e-5},
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    show_progress_bar = True
)

2022-12-20 17:53:07 - Warmup-steps: 2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/344 [00:00<?, ?it/s]

In [43]:
newlabel_cross_encoder = cross_encoder

In [42]:
newlabel_model_save_path 

'/home/sol3sts/train&dev_newlabel_cross_encoder_-2022-12-20_17-51-31'

### cross-encoder-test_evaluating

* 원래의 라벨로 train된 model과 새로운 라벨로 train된 model의 성능비교 

In [44]:
# og_label test 
print("OGlabel SPAERMAN R:",test_evaluator(original_klue_cross_encoder))

# new label test
print("NEWlabel SPAERMAN R:",test_evaluator(newlabel_cross_encoder))


OGlabel SPAERMAN R: 0.8960905803814517
NEWlabel SPAERMAN R: 0.9057458698857157


#### bin 별로 성능평가 

In [45]:
pairs_all = list(zip(test['sentence1'], test['sentence2']))

scores_all = newlabel_cross_encoder.predict(pairs_all,
                               show_progress_bar = True)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [46]:
test['silver_label'] = np.round((scores_all * 5).tolist(), 1)
# test['silver_label'] = (scores_all * 5).tolist()
test

,sentence1,sentence2,labels.label,silver_label
0,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",4.9,4.6
1,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,1.4,1.9
2,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,1.3,1.0
3,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",3.7,3.9
4,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,3.0,2.9
...,...,...,...,...
552,전자레인지말고 가스레인지를 쓰면 어떄?,믹서기 말고 가스레인지를 써보는 건 어때?,2.7,3.3
553,드럼 세탁기랑 통돌이 세탁기 세척력 비교해봐,드럼이랑 통돌이랑 세척력 좋은 것 부터 알려줘,3.4,3.5
554,깨끗하고 호스트의 배려가 돋보이는 숙소입니다.,그 외에는 호스트의 배려가 돋보이는 숙소였습니다.,3.4,3.9
555,"에어비앤비 광고메일이 더 많아, 트립 닷컴 메일이 더 많아?",에어비앤비랑 트립닷컴 중 누가 더 메일을 많이 보내?,3.4,3.5


In [47]:
print("'Spearman correlation' for whole test data:",test.corr(method='spearman')['silver_label'][0])
print("'Pearson correlation' for whole test data:",test.corr(method='pearson')['silver_label'][0])

'Spearman correlation' for whole test data: 0.9052453666405855
'Pearson correlation' for whole test data: 0.8954925424490879


In [48]:
correct=[]
for i in range(len(test)):
  if (test['silver_label'].values[i] >= 3.0) & (test['labels.label'].values[i] >= 3.0):
    correct.append('True_P')
  elif (test['silver_label'].values[i] < 3.0) & (test['labels.label'].values[i] < 3.0):
    correct.append('True_N')
  elif (test['silver_label'].values[i] >= 3.0) & (test['labels.label'].values[i] < 3.0):
    correct.append('False_P')
  else:
    correct.append('False_N')



In [49]:
test['correctness'] = correct

print("whole test data confusion matrix")
print()
print(test['correctness'].value_counts())

whole test data confusion matrix

True_N     239
True_P     222
False_P     84
False_N     12
Name: correctness, dtype: int64


In [50]:
# acc / f1 
TP = len(test[test['correctness'] == 'True_P'])
TN = len(test[test['correctness'] == 'True_N'])
FP = len(test[test['correctness'] == 'False_P'])
FN = len(test[test['correctness'] == 'False_N'])

print("precision", TP/(TP+FP))
print("recall", TP/(TP+FN))
pc=TP/(TP+FP)
rc=TP/(TP+FN)
print("f1 score", 2*pc*rc/(pc+rc))

precision 0.7254901960784313
recall 0.9487179487179487
f1 score 0.8222222222222223


0.5단위 bin 만들기

In [51]:
# train
t = []
for i in range(len(train)):
  if (train['labels.label'].values[i] >= 0.0)  & (train['labels.label'].values[i] <=0.2):
    t.append(0.0)
  elif (train['labels.label'].values[i] >= 0.2)  & (train['labels.label'].values[i] <=0.7):
    t.append(0.5)  
  elif (train['labels.label'].values[i] >= 0.8)  & (train['labels.label'].values[i] <=1.2):
    t.append(1.0)
  elif (train['labels.label'].values[i] >= 1.3)  & (train['labels.label'].values[i] <=1.7):
    t.append(1.5)  
  elif (train['labels.label'].values[i] >= 1.8)  & (train['labels.label'].values[i] <=2.2):
    t.append(2.0)
  elif (train['labels.label'].values[i] >= 2.3)  & (train['labels.label'].values[i] <=2.7):
    t.append(2.5)  
  elif (train['labels.label'].values[i] >= 2.8)  & (train['labels.label'].values[i] <=3.2):
    t.append(3.0)
  elif (train['labels.label'].values[i] >= 3.3)  & (train['labels.label'].values[i] <=3.7):
    t.append(3.5)  
  elif (train['labels.label'].values[i] >= 3.8)  & (train['labels.label'].values[i] <=4.2):
    t.append(4.0)
  elif (train['labels.label'].values[i] >= 4.3)  & (train['labels.label'].values[i] <=4.7):
    t.append(4.5)  
  elif (train['labels.label'].values[i] >= 4.8)  & (train['labels.label'].values[i] <=5.0):
    t.append(5.0)


print(len(t))

train['gold_bin'] = t


train_bin_05 = train[train['gold_bin'] == 0.5]
train_bin_10 = train[train['gold_bin'] == 1.0]
train_bin_15 = train[train['gold_bin'] == 1.5]
train_bin_20 = train[train['gold_bin'] == 2.0]
train_bin_25 = train[train['gold_bin'] == 2.5]
train_bin_30 = train[train['gold_bin'] == 3.0]
train_bin_35 = train[train['gold_bin'] == 3.5]
train_bin_40 = train[train['gold_bin'] == 4.0]
train_bin_45 = train[train['gold_bin'] == 4.5]
train_bin_50 = train[train['gold_bin'] == 5.0]



10992


In [52]:
# test
a = []
for i in range(len(test)):
  if (test['labels.label'].values[i] >= 0.0)  & (test['labels.label'].values[i] <=0.2):
    a.append(0.0)
  elif (test['labels.label'].values[i] >= 0.2)  & (test['labels.label'].values[i] <=0.7):
    a.append(0.5)  
  elif (test['labels.label'].values[i] >= 0.8)  & (test['labels.label'].values[i] <=1.2):
    a.append(1.0)
  elif (test['labels.label'].values[i] >= 1.3)  & (test['labels.label'].values[i] <=1.7):
    a.append(1.5)  
  elif (test['labels.label'].values[i] >= 1.8)  & (test['labels.label'].values[i] <=2.2):
    a.append(2.0)
  elif (test['labels.label'].values[i] >= 2.3)  & (test['labels.label'].values[i] <=2.7):
    a.append(2.5)  
  elif (test['labels.label'].values[i] >= 2.8)  & (test['labels.label'].values[i] <=3.2):
    a.append(3.0)
  elif (test['labels.label'].values[i] >= 3.3)  & (test['labels.label'].values[i] <=3.7):
    a.append(3.5)  
  elif (test['labels.label'].values[i] >= 3.8)  & (test['labels.label'].values[i] <=4.2):
    a.append(4.0)
  elif (test['labels.label'].values[i] >= 4.3)  & (test['labels.label'].values[i] <=4.7):
    a.append(4.5)  
  elif (test['labels.label'].values[i] >= 4.8)  & (test['labels.label'].values[i] <=5.0):
    a.append(5.0)

b = []
for i in range(len(test)):
  if (test['silver_label'].values[i] >= 0.0)  & (test['silver_label'].values[i] <=0.2):
    b.append(0.0)
  elif (test['silver_label'].values[i] >= 0.3)  & (test['silver_label'].values[i] <=0.7):
    b.append(0.5)  
  elif (test['silver_label'].values[i] >= 0.8)  & (test['silver_label'].values[i] <=1.2):
    b.append(1.0)
  elif (test['silver_label'].values[i] >= 1.3)  & (test['silver_label'].values[i] <=1.7):
    b.append(1.5)  
  elif (test['silver_label'].values[i] >= 1.8)  & (test['silver_label'].values[i] <=2.2):
    b.append(2.0)
  elif (test['silver_label'].values[i] >= 2.3)  & (test['silver_label'].values[i] <=2.7):
    b.append(2.5)  
  elif (test['silver_label'].values[i] >= 2.8)  & (test['silver_label'].values[i] <=3.2):
    b.append(3.0)
  elif (test['silver_label'].values[i] >= 3.3)  & (test['silver_label'].values[i] <=3.7):
    b.append(3.5)  
  elif (test['silver_label'].values[i] >= 3.8)  & (test['silver_label'].values[i] <=4.2):
    b.append(4.0)
  elif (test['silver_label'].values[i] >= 4.3)  & (test['silver_label'].values[i] <=4.7):
    b.append(4.5)  
  elif (test['silver_label'].values[i] >= 4.8)  & (test['silver_label'].values[i] <=5.0):
    b.append(5.0)

len(a), len(b)

(557, 557)

In [53]:

#test

test['differ']= np.round(test['labels.label'] - test['silver_label'],1)
test['gold_bin'] = a
test['silver_bin'] = b

test['same_bin']= (test['gold_bin'] == test['silver_bin'])

# bin간의 거리
test['bin_distance'] = test['gold_bin'] - test['silver_bin']


#MSE
test['SE'] = test['differ']**2 
print("MSE:",sum(test['SE']) / len(test)) 

# make bin
bin_05 = test[test['gold_bin'] == 0.5]
bin_10 = test[test['gold_bin'] == 1.0]
bin_15 = test[test['gold_bin'] == 1.5]
bin_20 = test[test['gold_bin'] == 2.0]
bin_25 = test[test['gold_bin'] == 2.5]
bin_30 = test[test['gold_bin'] == 3.0]
bin_35 = test[test['gold_bin'] == 3.5]
bin_40 = test[test['gold_bin'] == 4.0]
bin_45 = test[test['gold_bin'] == 4.5]
bin_50 = test[test['gold_bin'] == 5.0]

MSE: 0.46026929982046677


In [54]:
print("Test_Evaluation_Per_Bin")
print( "len = number of test_data,", "ME = mean_error")
print()
print("<bin_0.5>:","pearson:",bin_05.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_05.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_05), "MSE:",round(sum(bin_05['SE']) / len(bin_05),2),"ME:",round(sum(bin_05['differ']) / len(bin_05),2),"len(train):",len(train_bin_05))
print("<bin_1.0>:","pearson:",bin_10.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_10.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_10), "MSE:",round(sum(bin_10['SE']) / len(bin_10),2),"ME:",round(sum(bin_10['differ']) / len(bin_10),2),"len(train):",len(train_bin_10))
print("<bin_1.5>:","pearson:",bin_15.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_15.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_15), "MSE:",round(sum(bin_15['SE']) / len(bin_15),2),"ME:",round(sum(bin_15['differ']) / len(bin_15),2),"len(train):",len(train_bin_15))
print("<bin_2.0>:","pearson:",bin_20.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_20.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_20), "MSE:",round(sum(bin_20['SE']) / len(bin_20),2),"ME:",round(sum(bin_20['differ']) / len(bin_20),2),"len(train):",len(train_bin_20))
print("<bin_2.5>:","pearson:",bin_25.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_25.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_25), "MSE:",round(sum(bin_25['SE']) / len(bin_25),2),"ME:",round(sum(bin_25['differ']) / len(bin_25),2),"len(train):",len(train_bin_25))
print("<bin_3.0>:","pearson:",bin_30.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_30.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_30), "MSE:",round(sum(bin_30['SE']) / len(bin_30),2),"ME:",round(sum(bin_30['differ']) / len(bin_30),2),"len(train):",len(train_bin_30))
print("<bin_3.5>:","pearson:",bin_35.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_35.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_35), "MSE:",round(sum(bin_35['SE']) / len(bin_35),2),"ME:",round(sum(bin_35['differ']) / len(bin_35),2),"len(train):",len(train_bin_35))
print("<bin_4.0>:","pearson:",bin_40.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_40.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_40), "MSE:",round(sum(bin_40['SE']) / len(bin_40),2),"ME:",round(sum(bin_40['differ']) / len(bin_40),2),"len(train):",len(train_bin_40))
print("<bin_4.5>:","pearson:",bin_45.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_45.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_45), "MSE:",round(sum(bin_45['SE']) / len(bin_45),2),"ME:",round(sum(bin_45['differ']) / len(bin_45),2),"len(train):",len(train_bin_45))
print("<bin_5.0>:","pearson:",bin_50.corr(method='pearson')['silver_label'][0].round(2),"spaerman:",bin_50.corr(method='spearman')['silver_label'][0].round(2), "len:",len(bin_50), "MSE:",round(sum(bin_50['SE']) / len(bin_50),2),"ME:",round(sum(bin_50['differ']) / len(bin_50),2),"len(train):",len(train_bin_50))

Test_Evaluation_Per_Bin
len = number of test_data, ME = mean_error

<bin_0.5>: pearson: 0.11 spaerman: 0.16 len: 55 MSE: 0.61 ME: -0.41 len(train): 1079
<bin_1.0>: pearson: -0.03 spaerman: 0.0 len: 53 MSE: 0.53 ME: -0.25 len(train): 520
<bin_1.5>: pearson: 0.27 spaerman: 0.26 len: 46 MSE: 0.58 ME: -0.33 len(train): 374
<bin_2.0>: pearson: 0.06 spaerman: 0.05 len: 51 MSE: 0.93 ME: -0.56 len(train): 291
<bin_2.5>: pearson: 0.3 spaerman: 0.33 len: 63 MSE: 0.78 ME: -0.69 len(train): 302
<bin_3.0>: pearson: -0.34 spaerman: -0.27 len: 62 MSE: 0.44 ME: -0.4 len(train): 587
<bin_3.5>: pearson: 0.22 spaerman: 0.3 len: 62 MSE: 0.09 ME: -0.14 len(train): 1638
<bin_4.0>: pearson: 0.28 spaerman: 0.32 len: 55 MSE: 0.11 ME: 0.17 len(train): 2063
<bin_4.5>: pearson: 0.33 spaerman: 0.46 len: 55 MSE: 0.26 ME: 0.41 len(train): 1090
<bin_5.0>: pearson: 0.62 spaerman: 0.68 len: 22 MSE: 0.24 ME: 0.45 len(train): 104


In [55]:
# Confusion matrix

# acc / f1

def confusion_matrix(test):

  TP = len(test[test['correctness'] == 'True_P'])
  TN = len(test[test['correctness'] == 'True_N'])
  FP = len(test[test['correctness'] == 'False_P'])
  FN = len(test[test['correctness'] == 'False_N'])
  print("TP:",TP,"TN:",TN,"FP:",FP,"FN",FN)

  
  if (TP + FP) & (TP+FN) ==0:
    return("accuracy:",(TP+TN)/(TP+TN+FP+FN))
  
  else:
    pc=TP/(TP+FP)
    rc=TP/(TP+FN)
    
    return("accuracy:",(TP+TN)/(TP+TN+FP+FN),"precision:", TP/(TP+FP), "recall:", TP/(TP+FN),"f1 score:", 2*pc*rc/(pc+rc))

print("전체 test data")
print()
print("TP = 유사한페어를 유사하다고 정답,", "TN = 유사하지않은 페어를 유사하지 않다고 정답,", "FP=유사하지 않은페어를 유사하다고 오판,", "FN = 유사한페어를 유사하지않다고 오판")
print()
print("1.8-2.2:")
print(confusion_matrix(bin_20))
print()
print("2.3-2.7:")
print(confusion_matrix(bin_25))
print()
print("2.8-3.2:")
print(confusion_matrix(bin_30))
print()
print("3.3-3.7:")
print(confusion_matrix(bin_35))
print()
print("3.8-4.2:")
print(confusion_matrix(bin_40))
print()
print("4.3-4.7:")
print(confusion_matrix(bin_45))
print()
print("4.8-5.0:")
print(confusion_matrix(bin_50))

전체 test data

TP = 유사한페어를 유사하다고 정답, TN = 유사하지않은 페어를 유사하지 않다고 정답, FP=유사하지 않은페어를 유사하다고 오판, FN = 유사한페어를 유사하지않다고 오판

1.8-2.2:
TP: 0 TN: 35 FP: 16 FN 0
('accuracy:', 0.6862745098039216)

2.3-2.7:
TP: 0 TN: 18 FP: 45 FN 0
('accuracy:', 0.2857142857142857)

2.8-3.2:
TP: 31 TN: 1 FP: 21 FN 9
('accuracy:', 0.5161290322580645, 'precision:', 0.5961538461538461, 'recall:', 0.775, 'f1 score:', 0.6739130434782609)

3.3-3.7:
TP: 61 TN: 0 FP: 0 FN 1
('accuracy:', 0.9838709677419355, 'precision:', 1.0, 'recall:', 0.9838709677419355, 'f1 score:', 0.991869918699187)

3.8-4.2:
TP: 54 TN: 0 FP: 0 FN 1
('accuracy:', 0.9818181818181818, 'precision:', 1.0, 'recall:', 0.9818181818181818, 'f1 score:', 0.9908256880733944)

4.3-4.7:
TP: 54 TN: 0 FP: 0 FN 1
('accuracy:', 0.9818181818181818, 'precision:', 1.0, 'recall:', 0.9818181818181818, 'f1 score:', 0.9908256880733944)

4.8-5.0:
TP: 22 TN: 0 FP: 0 FN 0
('accuracy:', 1.0, 'precision:', 1.0, 'recall:', 1.0, 'f1 score:', 1.0)
